In [1]:
import tensorflow as tf
from onnx_tf.frontend import tensorflow_graph_to_onnx_model

In [ ]:
# Proto and graph freezing defined from
# https://github.com/onnx/tutorials/blob/master/tutorials/OnnxTensorflowExport.ipynb

# Bazel command below should find output node:
# bazel build C:\Users\camp-pry\AppData\Local\Continuum\anaconda3\envs\tensorflow_iris\Lib\site-packages\tensorflow\tools\graph_transforms:summarize_graph
#    bazel-bin/tensorflow/tools/graph_transforms/summarize_graph 
#    --in_graph=C:\Users\camp-pry\IdeaProjects\TensorflowTest\src\main\java\com\TensorflowTest\TensorflowTest\Training\stored_model\1533207083\saved_model.pb

# Bazel command below should generate new frozen model:
#bazel build tensorflow/python/tools:freeze_graph
#bazel-bin/tensorflow/python/tools/freeze_graph \
#    --input_graph=/home/mnist-tf/graph.proto \
#    --input_checkpoint=/home/mnist-tf/ckpt/model.ckpt \
#    --output_graph=/tmp/frozen_graph.pb \
#    --output_node_names=fc2/add \
#    --input_binary=True

In [15]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Exposes the Python wrapper for graph transforms."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# pylint: disable=unused-import,wildcard-import, line-too-long
from tensorflow.core.framework import graph_pb2
from tensorflow.python.framework import errors
from tensorflow.python.pywrap_tensorflow import TransformGraphWithStringInputs
from tensorflow.python.util import compat


def TransformGraph(input_graph_def, inputs, outputs, transforms):
  """Python wrapper for the Graph Transform Tool.

  Gives access to all graph transforms available through the command line tool.
  See documentation at https://github.com/tensorflow/tensorflow/blob/master/tensorflow/tools/graph_transforms/README.md
  for full details of the options available.

  Args:
    input_graph_def: GraphDef object containing a model to be transformed.
    inputs: List of node names for the model inputs.
    outputs: List of node names for the model outputs.
    transforms: List of strings containing transform names and parameters.

  Returns:
    New GraphDef with transforms applied.
  """

  input_graph_def_string = input_graph_def.SerializeToString()
  inputs_string = compat.as_bytes(",".join(inputs))
  outputs_string = compat.as_bytes(",".join(outputs))
  transforms_string = compat.as_bytes(" ".join(transforms))
  with errors.raise_exception_on_not_ok_status() as status:
    output_graph_def_string = TransformGraphWithStringInputs(
        input_graph_def_string, inputs_string, outputs_string,
        transforms_string, status)
  output_graph_def = graph_pb2.GraphDef()
  output_graph_def.ParseFromString(output_graph_def_string)
  return output_graph_def


In [25]:
# Converts frozen model to ONNX format
with tf.gfile.GFile("stored_model/1533207083/saved_model.pb", "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    onnx_model = tensorflow_graph_to_onnx_model(graph_def,
                                     "outputs", #This is the output node
                                     opset=6)

    file = open("models/iris/graph.onnx", "wb")
    file.write(onnx_model.SerializeToString())
    file.close()

DecodeError: Error parsing message